# Continued pretraining (CP) using Unsloth

In [1]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install --upgrade unsloth_zoo
%pip install wandb

Found existing installation: unsloth 2025.2.15
Uninstalling unsloth-2025.2.15:
  Successfully uninstalled unsloth-2025.2.15
  Cloning https://github.com/unslothai/unsloth.git to /data/temporary/pip-req-build-jljbsugj
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /data/temporary/pip-req-build-jljbsugj
  Resolved https://github.com/unslothai/unsloth.git to commit 14c9be1d7160162a90ce7a9a6cae36965563a0e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.2.15-py3-none-any.whl size=188380 sha256=acacb3407993170d5a87e1498e4d9627227e9d9637b951ceddd68c95d01ab546
  Stored in directory: /data/temporary/pip-ephem-wheel-cache-d5nc_pez/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pi

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from unsloth import UnslothTrainingArguments, UnslothTrainer
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [3]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Init LoRA

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"], #now also embeddings and lm_head
    lora_alpha = 512,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = True,  # We support rank stabilized LoRA
    # init_lora_weights = "loftq",
    # loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/_utils.py:760: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## Prepare Data

In [5]:
from datasets import load_dataset

# this dataset has already fixed encoding using ftfy (as is used by me in the preprocessing steps of other datasets)
dataset = load_dataset("HuggingFaceFW/fineweb-2", "ces_Latn")
#we need only texts
dataset = dataset.remove_columns(["id", "dump", "url", "date", "file_path", "language", "language_score", "language_script", "minhash_cluster_size", "top_langs"])
#shuffle to be sure we select "random sample"
dataset = dataset.shuffle(seed=42)
dataset

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/473 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 26631
    })
    train: Dataset({
        features: ['text'],
        num_rows: 62703458
    })
})

In [6]:
dataset["train"] = dataset["train"].select(range(1000))

In [7]:
def preprocess_function(examples):
    return {"text": [example + tokenizer.eos_token for example in examples["text"]]}

dataset = dataset.map(preprocess_function, batched=True)

In [8]:
for row in dataset["train"][:5]["text"]:
    print("=========================")
    print(row)

V dnešní době je wifi připojení doma či v práci nezbytností. Ať už pracujete z domova, streamujete filmy nebo hrajete online hry, stabilní a rychlé připojení je klíčem k úspěchu. Frekvence wifi signálu je jedním z faktorů, který může ovlivnit rychlost a stabilitu připojení. V tomto článku se zaměříme na pět kroků, které vám pomohou optimálně nastavit frekvenci vašeho wifi signálu.
Krok 1: Vyberte správný kanál
Naše wifi sítě fungují na dvou frekvencích – 2,4 GHz a 5 GHz. Každá z těchto frekvencí má určité množství kanálů. Na frekvenci 2,4 GHz máme k dispozici 14 kanálů. Kanály jsou však překrývající se, což znamená, že pokud jsou dva sousedící kanály použity, mohou si vzájemně rušit. Proto se v praxi používají převážně kanály 1, 6 a 11.
Na frekvenci 5 GHz je k dispozici více kanálů a ty jsou navíc nezávislé, tedy se nepřekrývají. To znamená, že na této frekvenci můžeme dosáhnout vyšší přenosové rychlosti.
Krok 2: Změřte rušení
Pokud chcete zjistit, který kanál je pro vás nejlepší, může

## Train the model

In [9]:
use_kl_loss = True

In [10]:
class UnslothKLLossTrainer(UnslothTrainer):
    def __init__(self, ref_model, kl_weight=0.1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ref_model = ref_model
        self.ref_model.eval()
        self.kl_weight = kl_weight

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        #account for the actual number of items in the batch
        if num_items_in_batch is not None:
            effective_batch_size = num_items_in_batch
        else:
            effective_batch_size = labels.size(0)
        
        #cross entropy loss
        loss_fct = torch.nn.CrossEntropyLoss(reduction="sum")
        loss_ce = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
        loss_ce = loss_ce / effective_batch_size # Scale by actual number of items

        # KL loss
        with torch.no_grad():
            ref_outputs = self.ref_model(**inputs)
            ref_logits = ref_outputs.logits
        
        loss_kl_fct = torch.nn.KLDivLoss(reduction="sum")
        loss_kl = loss_kl_fct(
            torch.nn.functional.log_softmax(logits, dim=-1),
            torch.nn.functional.softmax(ref_logits, dim=-1),
        )
        loss_kl = loss_kl / effective_batch_size # Scale by actual number of items

        total_loss = loss_ce + self.kl_weight*loss_kl

        return (total_loss, outputs) if return_outputs else total_loss


if use_kl_loss:
    TrainerClass = UnslothKLLossTrainer
else:
    TrainerClass = UnslothTrainer

In [11]:
import copy

trainer = TrainerClass(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    ref_model=copy.deepcopy(model),
    
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 1000,
        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = SEED,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        run_name="llama3.2-3b-instruct-cont_pretrain_example",
        # eval_strategy = "steps",
        # eval_steps = 50,
        
    ),
)

/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:578: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:592: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:606: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any val

In [12]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.381 GB.
10.312 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 17
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 1,177,026,560


Unsloth: Setting lr = 5.00e-06 instead of 5.00e-05 for embed_tokens.
Unsloth: Setting lr = 5.00e-06 instead of 5.00e-05 for lm_head.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlynatom. Use `wandb login --relogin` to force relogin


/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/llama.py:1891: SyntaxWarning: invalid escape sequence '\.'
  start = re.search('logger\.info\([\"\'].+?Running training', inner_training_loop).span(0)[0]
/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/llama.py:1894: SyntaxWarning: invalid escape sequence '\s'
  spaces = re.search('\n([\s\t]{1,})', original_debug).group(0)[1:]
/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/llama.py:1895: SyntaxWarning: invalid escape sequence '\s'
  front_spaces = re.match('([\s\t]{1,})', inner_training_loop).group(0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.50 GiB. GPU 0 has a total capacity of 39.38 GiB of which 1.83 GiB is free. Including non-PyTorch memory, this process has 37.53 GiB memory in use. Of the allocated memory 36.69 GiB is allocated by PyTorch, and 344.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.
